In [3]:
import json
import numpy as np
from statsmodels.stats.inter_rater import fleiss_kappa

# Load json
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Adjust image IDs by a specified amount because of id not matching error
def adjust_image_ids(data, adjust_value):
    for entry in data:
        entry['id'] -= adjust_value  # Adjust the image ID by the specified value
    return data

# Save the modified JSON data
def save_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, indent=4)

# Sync annotations from multiple annotators
def sync_annotations(annotator_files):
    """
    Sync annotations by image ID across various annotators.
    """
    synchronized_data = {}
    for file_path in annotator_files:
        data = load_json(file_path)
        print(f"Loaded data from {file_path}")

        for entry in data:
            img_id = entry['id']
            annotations = entry.get('annotations', [])
            if annotations and 'result' in annotations[0]:
                result = annotations[0]['result']
                if result and 'value' in result[0] and 'choices' in result[0]['value']:
                    label = result[0]['value']['choices'][0]
                    synchronized_data.setdefault(img_id, []).append(label)
                    print(f"ID: {img_id}, Label: {label}")

    return synchronized_data

# Ensure consistent annotations from all annotators
def filter_consistent_annotations(synchronized_data, annotator_count):

    return {img_id: annotations for img_id, annotations in synchronized_data.items() if len(annotations) == annotator_count}

# Prepare the Fleiss' Kappa matrix
def prepare_fleiss_kappa_matrix(synchronized_data, categories):
    """
    Create the Fleiss' Kappa matrix from synchronized annotations.
    """
    matrix = np.zeros((len(synchronized_data), len(categories)), dtype=int)
    for i, (img_id, annotations) in enumerate(synchronized_data.items()):
        for label in annotations:
            if label in categories:
                matrix[i, categories.index(label)] += 1
    return matrix


# Annotator files
annotator_files = [
        '/content/CV data1.json',
        '/content/CV data2.json',
        '/content/CV data3.json'
    ]

# Adjust image IDs for the first two files and save
adjust_value_a1 = 43260
adjust_value_a2 = 14398

# Adjust image IDs for 'a1' and 'a2' files and save them
data_a1 = adjust_image_ids(load_json(annotator_files[0]), adjust_value_a1)
save_json(data_a1, 'cv_data1_adjusted.json')
print(f"Updated image IDs in 'a1.json' by reducing them by {adjust_value_a1}.")

data_a2 = adjust_image_ids(load_json(annotator_files[1]), adjust_value_a2)
save_json(data_a2, 'cv_data2_adjusted.json')
print(f"Updated image IDs in 'a2.json' by reducing them by {adjust_value_a2}.")

# Load and sync annotations from all annotator files
synced_data = sync_annotations(['cv_data1_adjusted.json', 'cv_data2_adjusted.json', annotator_files[2]])

# Filter consistent annotations (all annotators must annotate the same image)
annotator_count = len(annotator_files)
synced_data = filter_consistent_annotations(synced_data, annotator_count)

# If no consistent data, print a warning and exit
if not synced_data:
    print("No consistent data found (some images don't have annotations from all annotators).")
else:
    categories = ["Trucks", "No Trucks"]
    matrix = prepare_fleiss_kappa_matrix(synced_data, categories)
    print("Fleiss' Kappa Matrix:")
    print(matrix)

    # Calculate Fleiss' Kappa
    if matrix.shape[1] < 2 or np.all(np.sum(matrix, axis=1) <= 1):
        print("Insufficient data: Fleiss' Kappa requires multiple annotations per image.")
    else:
        kappa = fleiss_kappa(matrix)
        print(f"Fleiss' Kappa: {kappa:.4f}")


Updated image IDs in 'a1.json' by reducing them by 43260.
Updated image IDs in 'a2.json' by reducing them by 14398.
Loaded data from cv_data1_adjusted.json
ID: 41, Label: Trucks
ID: 42, Label: No Trucks
ID: 43, Label: No Trucks
ID: 44, Label: No Trucks
ID: 45, Label: Trucks
ID: 46, Label: Trucks
ID: 47, Label: No Trucks
ID: 48, Label: Trucks
ID: 49, Label: No Trucks
ID: 50, Label: Trucks
ID: 51, Label: No Trucks
ID: 52, Label: Trucks
ID: 53, Label: Trucks
ID: 54, Label: No Trucks
ID: 55, Label: Trucks
ID: 56, Label: Trucks
ID: 57, Label: Trucks
ID: 58, Label: Trucks
ID: 59, Label: No Trucks
ID: 60, Label: Trucks
Loaded data from cv_data2_adjusted.json
ID: 41, Label: Trucks
ID: 42, Label: No Trucks
ID: 43, Label: No Trucks
ID: 44, Label: No Trucks
ID: 45, Label: No Trucks
ID: 46, Label: Trucks
ID: 47, Label: No Trucks
ID: 48, Label: No Trucks
ID: 49, Label: No Trucks
ID: 50, Label: Trucks
ID: 51, Label: No Trucks
ID: 52, Label: Trucks
ID: 53, Label: No Trucks
ID: 54, Label: No Trucks
ID

Fleiss kappa score of 0.7998 indicate that the annotators have labelled the data almost in the same way but whereas there are some differences which might be due to some misunderstandings of the data such as there were trucks hidden in the background which would normally be not clearly visible to some annotators which has led to that fleiss kappa score.

In [6]:
import json
from sklearn.metrics import cohen_kappa_score

# Load json
file_path_annotator_1 = "/content/updated_nlp_data1.json"
file_path_annotator_2 = "/content/updated_nlp_data2.json"

with open(file_path_annotator_1, "r") as file1, open(file_path_annotator_2, "r") as file2:
    data_annotator_1 = json.load(file1)
    data_annotator_2 = json.load(file2)

annotator_1_labels = []
annotator_2_labels = []

#function to extract labels
def extract_label(task):
    try:
        return task["annotations"][0]["result"][0]["value"]["labels"][0]
    except (IndexError, KeyError):
        return None

# Create mappings for task IDs to labels
annotations_1 = {task["id"]: extract_label(task) for task in data_annotator_1}
annotations_2 = {task["id"]: extract_label(task) for task in data_annotator_2}

# Match tasks and collect labels for common task IDs
common_task_ids = set(annotations_1.keys()) & set(annotations_2.keys())
for task_id in common_task_ids:
    label_1 = annotations_1[task_id]
    label_2 = annotations_2[task_id]
    if label_1 is not None and label_2 is not None:
        annotator_1_labels.append(label_1)
        annotator_2_labels.append(label_2)

# Calculate Cohen's kappa
if annotator_1_labels and annotator_2_labels:
    kappa = cohen_kappa_score(annotator_1_labels, annotator_2_labels)
    print(f"Cohen's kappa: {kappa}")
else:
    print("No matching tasks with valid labels found or not enough data for calculation.")


Cohen's kappa: 0.7297297297297298


Cohen's kappa score of 0.7297 indicates that both the annotators are thinking in the same way about the POS tags but there may be some misunderstandings such as should the word be considered as PROPN or NOUN and so on which led to that cohen's kappa score.

In [4]:
# this code is used for matching the ids in the json file

import json

# Load json
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# Save the updated json
def save_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4)

# Adjust image IDs
def adjust_image_ids(data, adjusted_val):
    for item in data:
        item['id'] -= adjusted_val
    return data

file_path_1 = '/content/NLP data1.json'
file_path_2 = '/content/NLP data2.json'

# Load data
data1 = load_json(file_path_1)
data2 = load_json(file_path_2)

min_id_1 = min(item['id'] for item in data1)
min_id_2 = min(item['id'] for item in data2)

# Choose the smaller of the two minimums
adjusted_val = min(min_id_1, min_id_2)

# Reduce IDs
updated_data1 = adjust_image_ids(data1, 14957)
updated_data2 = adjust_image_ids(data2, 0)

# Save the updated data back into separate JSON files
save_json(updated_data1, 'updated_nlp_data1.json')
save_json(updated_data2, 'updated_nlp_data2.json')

print(f"IDs in both files have been reduced by {reduction_value}.")
print("Updated annotations saved to 'updated_nlp_data1.json' and 'updated_nlp_data2.json'.")


IDs in both files have been reduced by 14460.
Updated annotations saved to 'updated_nlp_data1.json' and 'updated_nlp_data2.json'.


this code was used for updating the ids in the annotations present in the json file because there are errors occuring due to mismatching of ids in the json files